In [16]:
import pandas as pd
import os,sys, imp, glob
sys.path.append('./src/')
import doc_utils_clean as doc_utils
import pos_yap_process
import feature_utils
import common_utils
import defines
import numpy as np
import re

In [22]:
doc_list =  glob.glob(os.path.join(os.path.join(os.getcwd(),"tmp","agreement",r"*_Compare_*.docx")))
doc_list.sort()
doc_map = {}

for idx,doc in enumerate(doc_list):
    f_name=re.sub('.docx','',os.path.basename(doc).split('_')[-1])
    doc_map[f_name] = doc
doc_map

{'Dana': '/scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/tmp/agreement/01_Compare_Dana.docx',
 'Mor': '/scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/tmp/agreement/02_Compare_Mor.docx'}

In [23]:
dir_name="agreement"

In [25]:
imp.reload(doc_utils)
doc_utils.save_docs_db(dir_name,doc_list)

Creating doc_db


In [59]:
doc_db=pd.read_csv(os.path.join(os.getcwd(), defines.PATH_TO_DFS, dir_name, "doc_db.csv"))
doc_db

,path,file_name,doc_idx_from_name
0,/scratch/sim_reg6/users/zsofya/classroom/MSc/p...,01_Compare_Dana.docx,1
1,/scratch/sim_reg6/users/zsofya/classroom/MSc/p...,02_Compare_Mor.docx,2


In [35]:
dir_name

'agreement'

In [89]:
imp.reload(doc_utils)
doc_utils.parse_all_docs(dir_name=dir_name,merge_short_sent=False,doc_path_list=doc_list)

Creating doc_db
1 Doc 587 sentences saved
Finished doc 0 of 2
2 Doc 563 sentences saved
Finished doc 1 of 2


In [90]:
db = {}
for db_type in ['par_db','sent_db']:
    db[db_type]={}
    for idx in doc_db['doc_idx_from_name'].values:
        db[db_type][idx]=pd.read_csv(os.path.join(os.getcwd(), defines.PATH_TO_DFS, dir_name, "{:02d}_{}.csv".format(idx,db_type)))

In [91]:
for key,value in db.items():
    print(key,len(value[1]),len(value[2]))


par_db 164 164
sent_db 587 563


In [92]:
sent_count={}
sent_count[1]=db['sent_db'][1].groupby('par_idx_in_doc').size()

In [93]:
sent_count[2]=db['sent_db'][2].groupby('par_idx_in_doc').size()

In [94]:
sent_count[2]

par_idx_in_doc
0.000000      1
1.000000      1
2.000000      2
3.000000      1
4.000000      1
             ..
159.000000    1
160.000000    2
161.000000    1
162.000000    8
163.000000    8
Length: 160, dtype: int64

In [95]:
db_diff = pd.DataFrame()
db_diff['1']=sent_count[1]
db_diff['2']=sent_count[2]
db_diff

,1,2
par_idx_in_doc,,
0.000000,1,1
1.000000,1,1
2.000000,2,2
3.000000,1,1
4.000000,1,1
...,...,...
159.000000,1,1
160.000000,2,2
161.000000,1,1


In [96]:
diff_par_indices=db_diff[db_diff['1']!=db_diff['2']].index

In [97]:
db_diff[db_diff['1']!=db_diff['2']]

,1,2
par_idx_in_doc,,
31.000000,19,4
43.000000,11,12
47.000000,27,26
49.000000,6,10
55.000000,7,8
71.000000,9,10
101.000000,15,9
109.000000,5,7
123.000000,8,7


In [104]:
db['sent_db'][1].query("par_idx_in_doc.isin(@diff_par_indices)")

,is_question,text,sent_idx_in_block,block_idx,is_nar,doc_idx,par_db_idx,par_idx_in_doc,par_type,block_type,nar_idx,sent_len,is_client,sent_idx_in_par,sent_pos_in_par,sent_pos_in_doc
40,1.000000,אז אמ כי זה שוב ממש דיברנו על התדמית מאוד שי...,0.000000,32.000000,0.000000,1.000000,31.000000,31.000000,client,not_nar,0.000000,69.000000,1,0,0.052632,0.069847
41,0.000000,מאוד משפיע על המצברוח שלי אם אני עכשיו אני מבי...,1.000000,32.000000,0.000000,1.000000,31.000000,31.000000,client,not_nar,0.000000,83.000000,1,1,0.105263,0.071550
42,0.000000,עשיתי על זה כבר עבודה על זה עם הפסיכולוגית הקו...,0.000000,33.000000,1.000000,1.000000,31.000000,31.000000,client,whole,2.000000,65.000000,1,2,0.157895,0.073254
43,0.000000,הרבה,1.000000,33.000000,1.000000,1.000000,31.000000,31.000000,client,whole,2.000000,4.000000,1,3,0.210526,0.074957
44,0.000000,אבל אה אפרופו תדמיות,2.000000,33.000000,1.000000,1.000000,31.000000,31.000000,client,whole,2.000000,20.000000,1,4,0.263158,0.076661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,1.000000,איך זה?,9.000000,167.000000,1.000000,1.000000,144.000000,144.000000,client,whole,22.000000,7.000000,1,12,0.928571,0.877342
515,1.000000,איך זה יעבור?,10.000000,167.000000,1.000000,1.000000,144.000000,144.000000,client,whole,22.000000,13.000000,1,13,1.000000,0.879046
564,0.000000,לא מאה אחוז מה אתה מרגיש אני גם – אני גם שומר ...,0.000000,182.000000,0.000000,1.000000,158.000000,158.000000,client,not_nar,0.000000,362.000000,1,0,0.333333,0.962521
565,0.000000,הוא אמר לי – אם אתה היית במקומי אתה היית הולך,0.000000,183.000000,1.000000,1.000000,158.000000,158.000000,client,whole,24.000000,45.000000,1,1,0.666667,0.964225


In [105]:
db['sent_db'][2].query("par_idx_in_doc.isin(@diff_par_indices)")

,is_question,text,sent_idx_in_block,block_idx,is_nar,doc_idx,par_db_idx,par_idx_in_doc,par_type,block_type,nar_idx,sent_len,is_client,sent_idx_in_par,sent_pos_in_par,sent_pos_in_doc
40,0.000000,אז אמ כי זה שוב ממש דיברנו על התדמית,0.000000,32.000000,0.000000,2.000000,31.000000,31.000000,client,not_nar,0.000000,36.000000,1,0,0.250000,0.072824
41,0.000000,מאוד זה קצת עבר לי,1.000000,32.000000,0.000000,2.000000,31.000000,31.000000,client,not_nar,0.000000,19.000000,1,1,0.500000,0.074600
42,0.000000,זה ו זהו זה גם בדרך כלל קורה לי לפני היום הולדת,2.000000,32.000000,0.000000,2.000000,31.000000,31.000000,client,not_nar,0.000000,47.000000,1,2,0.750000,0.076377
43,0.000000,יום שלישי הבא היום הולדת,3.000000,32.000000,0.000000,2.000000,31.000000,31.000000,client,not_nar,0.000000,24.000000,1,3,1.000000,0.078153
59,0.000000,יש אה אני מאמין ש אני מאמין שיש איזה מחזוריות ...,0.000000,45.000000,0.000000,2.000000,43.000000,43.000000,client,not_nar,0.000000,101.000000,1,0,0.083333,0.106572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,1.000000,אז מה?,5.000000,161.000000,1.000000,2.000000,144.000000,144.000000,client,whole,17.000000,6.000000,1,9,0.833333,0.872114
491,1.000000,איך זה?,6.000000,161.000000,1.000000,2.000000,144.000000,144.000000,client,whole,17.000000,7.000000,1,10,0.916667,0.873890
492,1.000000,איך זה יעבור?,7.000000,161.000000,1.000000,2.000000,144.000000,144.000000,client,whole,17.000000,13.000000,1,11,1.000000,0.875666
541,0.000000,לא מאה אחוז,0.000000,176.000000,0.000000,2.000000,158.000000,158.000000,client,not_nar,0.000000,11.000000,1,0,0.500000,0.962700
